In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [4]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
np.random.seed(0)
translator = CirqTranslater(5, untouchable_blocks = [])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

In [36]:
cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
circuit_db = pd.DataFrame(cdb)


In [37]:
circuit, circuit_db = translator.give_circuit(circuit_db)
circuit

┌──┐                                                               ┌──┐
(0, 0): ───@───Rz(th_0)───@───Rz(th_2)───@───Rz(th_4)───@───Rz(th_6)───X───Rx(th_9)─────────────────────X────Rx(th_17)─────────────────────────────────────────────────────X────Rx(th_25)───────────────────────────────────────────────────X───Rx(th_33)───────────────────────────────────────────────────
           │              │              │              │              │                                │                                                                  │                                                                │
(0, 1): ───X───Rx(th_1)───┼──────────────┼──────────────┼──────────────@───Rz(th_8)───@───Rz(th_10)────@┼────Rz(th_12)───@───Rz(th_14)───X───Rx(th_19)─────────────────────┼────────────────X───Rx(th_27)───────────────────────────────────┼───────────────X───Rx(th_35)───────────────────────────────────
                          │              │              │                             │                ││                │               │                                 │                │                                               │               │
(0, 2): ──────────────────X───Rx(th_3)───┼──────────────┼─────────────────────────────X───Rx(th_11)────┼@────Rz(th_16)───┼───────────────@───Rz(th_18)───@───Rz(th_20)────@┼────Rz(th_22)───┼───────────────X───Rx(th_29)───────────────────┼───────────────┼───────────────X───Rx(th_37)───────────────────
                                         │              │                                              │                 │                               │                ││                │               │                               │               │               │
(0, 3): ─────────────────────────────────X───Rx(th_5)───┼──────────────────────────────────────────────X─────Rx(th_13)───┼───────────────────────────────X───Rx(th_21)────┼@────Rz(th_24)───@───Rz(th_26)───@───Rz(th_28)───@───Rz(th_30)───┼───────────────┼───────────────┼───────────────X───Rx(th_39)───
                                                        │                                                                │                                                │                                                 │               │               │               │               │
(0, 4): ────────────────────────────────────────────────X───Rx(th_7)─────────────────────────────────────────────────────X───Rx(th_15)────────────────────────────────────X─────Rx(th_23)───────────────────────────────────X───Rx(th_31)───@───Rz(th_32)───@───Rz(th_34)───@───Rz(th_36)───@───Rz(th_38)───
                                                                                                      └──┘                                                               └──┘

In [38]:
simplifier = Simplifier(translator)
simplifier.relative_rules = [simplifier.rule_6]
simplifier.absolute_rules = [simplifier.rule_6]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
hey, i'm still simplifying, cnt8
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6d454e6a0>>
hey, i'm still simplifying, cnt9
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0

In [39]:
gates_on_qubit, on_qubit_order = simplifier.get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()

In [40]:
mdif = max_diff(translator , circuit_db, simplified_db)
mdif

0.0

In [41]:
simplified_db

,ind,symbol,param_value,trainable,block_id
0,0,None,NaN,True,0
1,20,th_0,0.548814,True,0
2,26,th_1,0.715189,True,0
3,1,None,NaN,True,0
4,20,th_2,0.602763,True,0
5,27,th_3,0.544883,True,0
6,2,None,NaN,True,0
7,20,th_4,0.423655,True,0
8,28,th_5,0.645894,True,0
9,3,None,NaN,True,0


In [43]:
circuit_db0 = concatenate_dbs([circuit_db, u2_layer(translator)])
c, cdb = translator.give_circuit(circuit_db0)
simplifier = Simplifier(translator)
simplifier.relative_rules = [simplifier.rule_6]
simplifier.absolute_rules = [simplifier.rule_6]
simplified_db, ns = simplifier.reduce_circuit(cdb)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
hey, i'm still simplifying, cnt8
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c78bf0f0>>
hey, i'm still simplifying, cnt9
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0

In [44]:
mdif = max_diff(translator , cdb, simplified_db)
mdif

6.39734408315113e-16

In [45]:
circuit_db0 = concatenate_dbs([u2_layer(translator), circuit_db])
c, cdb = translator.give_circuit(circuit_db0)
simplifier = Simplifier(translator)
simplifier.relative_rules = [simplifier.rule_6]
simplifier.absolute_rules = [simplifier.rule_6]
simplified_db, ns = simplifier.reduce_circuit(cdb)
ss, simplified_db = translator.give_circuit(simplified_db)
mdif = max_diff(translator , cdb, simplified_db)
mdif

warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_0
 symbols ['th_0', 'th_1', 'th_2', 'th_3', 'th_4', 'th_5', 'th_6', 'th_7', 'th_8', 'th_9', 'th_10', 'th_11', 'th_12', 'th_13', 'th_14', 'th_15', 'th_16', 'th_17', 'th_18', 'th_19', 'th_20', 'th_21', 'th_22', 'th_23', 'th_24', 'th_25', 'th_26', 'th_27', 'th_28', 'th_29', 'th_30', 'th_31', 'th_32', 'th_33', 'th_34', 'th_35', 'th_36', 'th_37', 'th_38', 'th_39', 'th_40', 'th_41', 'th_42', 'th_43', 'th_44', 'th_45', 'th_46', 'th_47', 'th_48', 'th_49', 'th_50', 'th_51', 'th_52', 'th_53', 'th_54', 'th_55', 'th_56', 'th_57', 'th_58', 'th_59', 'th_60', 'th_61', 'th_62', 'th_63', 'th_64', 'th_65', 'th_66', 'th_67', 'th_68', 'th_69', 'th_70', 'th_71', 'th_72', 'th_73', 'th_74']
circuit_db {0: {'ind': 20, 'symbol': 'th_0', 'param_value': 1.8738857142115568, 'trainable': True, 'block_id': 0}, 1: {'ind': 25, 'symbol': 'th_1', 'param_value': 8.33392837771417, 'trainable': True, 'block_id': 0}, 2: {'ind': 20, 'symb

warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_0
 symbols ['th_0']
circuit_db {0: {'ind': 20, 'symbol': 'th_0', 'param_value': 1.8738857142115568, 'trainable': True, 'block_id': 0}, 1: {'ind': 25, 'symbol': 'th_0', 'param_value': 8.33392837771417, 'trainable': True, 'block_id': 0}} 



warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_1
 symbols ['th_0', 'th_0', 'th_1']
circuit_db {0: {'ind': 20, 'symbol': 'th_0', 'param_value': 1.8738857142115568, 'trainable': True, 'block_id': 0}, 1: {'ind': 25, 'symbol': 'th_0', 'param_value': 8.33392837771417, 'trainable': True, 'block_id': 0}, 2: {'ind': 21, 'symbol': 'th_1', 'param_value': -0.9401815452333921, 'trainable': True, 'block_id': 0}, 3: {'ind': 26, 'symbol': 'th_1', 'param_value': -2.7341504025444046, 'trainable': True, 'block_id': 0}} 



warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_2
 symbols ['th_0', 'th_0', 'th_1',

<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7fe6c789c2b0>>
warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_0
 symbols ['th_0']
circuit_db {0: {'ind': 20, 'symbol': 'th_0', 'param_value': 1.8738857142115568, 'trainable': True, 'block_id': 0}, 1: {'ind': 25, 'symbol': 'th_0', 'param_value': 8.33392837771417, 'trainable': True, 'block_id': 0}} 



warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_1
 symbols ['th_0', 'th_0', 'th_1']
circuit_db {0: {'ind': 20, 'symbol': 'th_0', 'param_value': 1.8738857142115568, 'trainable': True, 'block_id': 0}, 1: {'ind': 25, 'symbol': 'th_0', 'param_value': 8.33392837771417, 'trainable': True, 'block_id': 0}, 2: {'ind': 21, 'symbol': 'th_1', 'param_value': -0.9401815452333921, 'trainable': True, 'block_id': 0}, 3: {'ind': 26, 'symbol': 'th_1', 'param_value': -2.7341504025444046, 'trainable': True, 'block_id': 0}} 



warning, repeated symbol wh

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



OK we have a problem houston